In [2]:
!git clone -b janina https://github.com/inttx/DLAM_SealedSurfaces.git
%cd DLAM_SealedSurfaces

Cloning into 'DLAM_SealedSurfaces'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 70 (delta 36), reused 43 (delta 20), pack-reused 0 (from 0)
Receiving objects: 100% (70/70), 19.22 KiB | 1.75 MiB/s, done.
Resolving deltas: 100% (36/36), done.
/home/janina/PycharmProjects/DLAM_SealedSurfaces/DLAM_SealedSurfaces


In [1]:
!pip install -r requirements.txt
%cd src


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
/home/janina/PycharmProjects/DLAM_SealedSurfaces/src


In [2]:
from torch.utils.data import DataLoader
from torch import nn
from torch.optim import AdamW

from settings import *
from dataset import PotsdamDataset, get_data_loaders
from models import custom_resnet18, baseline_deeplabv3plus_resnet101
from train import train_loop

In [3]:
try:
    import os
    import shutil
    from google.colab import drive
    MOUNTPOINT = '/content/drive/'
    drive.mount(MOUNTPOINT)

    IMAGE_PATH = os.path.join(MOUNTPOINT, 'MyDrive', 'DLAM', '2_Ortho_RGB') # TODO adjust to your path
    LABEL_PATH = os.path.join(MOUNTPOINT, 'MyDrive', 'DLAM', '5_Labels_all') # TODO adjust to your path
    
    local_img_path = '/content/2_Ortho_RGB'
    local_lbl_path = '/content/5_Labels_all'
    
    os.makedirs(local_img_path, exist_ok=True)
    os.makedirs(local_lbl_path, exist_ok=True)
    
    # Copy from Drive to local storage
    shutil.copytree(IMAGE_PATH, local_img_path, dirs_exist_ok=True)
    shutil.copytree(LABEL_PATH, local_lbl_path, dirs_exist_ok=True)
    
    IMAGE_PATH = '/content/2_Ortho_RGB'
    LABEL_PATH = '/content/5_Labels_all'

    SAVE_PATH = MOUNTPOINT + 'MyDrive/DLAM/models/'
except:
    IMAGE_PATH = '../data/2_Ortho_RGB' # TODO adjust to your path
    LABEL_PATH = '../data/5_Labels_all' # TODO adjust to your path
    SAVE_PATH = '../models/'

In [4]:
# Hyperparameters
patch_size = 250
stride = 250
batch_size = 8
num_epochs = 2
lr = 0.001

# Train resnet18

In [ ]:
dataset = PotsdamDataset(IMAGE_PATH, LABEL_PATH, patch_size=patch_size, stride=stride, device=DEVICE)
train_loader, val_loader, test_loader = get_data_loaders(dataset, [0.8, 0.1, 0.1] ,batch_size)

model = custom_resnet18(patch_size=patch_size, device=DEVICE)
loss_fn = nn.MSELoss()
optimizer = AdamW(model.parameters(), lr=lr)

train_loop(train_loader=train_loader, val_loader=val_loader, model=model, loss_fn=loss_fn, optimizer=optimizer, num_epochs=num_epochs, device=DEVICE, save_path=SAVE_PATH + 'resnet18.pth')

Training:  14%|█▍        | 309/2189 [03:20<20:05,  1.56it/s, loss=1.88e+4]

# Train DeepLabV3 resnet101 baseline

In [ ]:
dataset = PotsdamDataset(IMAGE_PATH, LABEL_PATH, patch_size=patch_size, stride=stride, device=DEVICE, no_color_labels=True)
train_loader, val_loader, test_loader = get_data_loaders(dataset, [0.8, 0.1, 0.1] ,batch_size)

model = baseline_deeplabv3plus_resnet101(num_classes=6, device=DEVICE)
loss_fn = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=lr)

train_loop(train_loader=train_loader, val_loader=val_loader, model=model, loss_fn=loss_fn, optimizer=optimizer, num_epochs=num_epochs, device=DEVICE, save_path=SAVE_PATH + 'deeplabv3_resnet101.pth')